In [11]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [12]:
df = pd.read_csv("./db/dvf.csv", sep=",", encoding="utf-8")

C:\Users\darkj\AppData\Local\Temp\ipykernel_33300\2506603767.py:1: DtypeWarning: Columns (10,12,14,16,17,18,20,22,24,26,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./db/dvf.csv", sep=",", encoding="utf-8")


In [13]:
print(df.columns)

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')


In [14]:
df = df[df['valeur_fonciere'].notnull()]

In [15]:
# df.to_csv('./db/dvf_clean.csv', index=False)

In [16]:
df_bordeaux = df[df['nom_commune'] == "Bordeaux"]
# print(df_bordeaux)
df_bordeaux.to_csv('./db/Bordeaux.csv', index=False)

In [17]:
df_etabl = pd.read_csv("./db/fr-en-annuaire-education.csv", sep=";", encoding="utf-8")

df_etabl_bordeaux = df_etabl[df_etabl['Nom_commune'] == "Bordeaux"]
df_etabl_bordeaux.to_csv('./db/educ_bordeaux.csv', index=False)

C:\Users\darkj\AppData\Local\Temp\ipykernel_33300\4007602168.py:1: DtypeWarning: Columns (8,10,18,19,20,39,41,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_etabl = pd.read_csv("./db/fr-en-annuaire-education.csv", sep=";", encoding="utf-8")


In [18]:
df_etabl_bordeaux = pd.read_csv("./db/educ_bordeaux.csv", sep=";", encoding="utf-8")
print(df_etabl_bordeaux.columns)

Index(['Identifiant_de_l_etablissement,Nom_etablissement,Type_etablissement,Statut_public_prive,Adresse_1,Adresse_2,Adresse_3,Code_postal,Code_commune,Nom_commune,Code_departement,Code_academie,Code_region,Ecole_maternelle,Ecole_elementaire,Voie_generale,Voie_technologique,Voie_professionnelle,Telephone,Fax,Web,Mail,Restauration,Hebergement,ULIS,Apprentissage,Segpa,Section_arts,Section_cinema,Section_theatre,Section_sport,Section_internationale,Section_europeenne,Lycee_Agricole,Lycee_militaire,Lycee_des_metiers,Post_BAC,Appartenance_Education_Prioritaire,GRETA,SIREN_SIRET,Nombre_d_eleves,Fiche_onisep,position,Type_contrat_prive,Libelle_departement,Libelle_academie,Libelle_region,coordX_origine,coordY_origine,epsg_origine,nom_circonscription,latitude,longitude,precision_localisation,date_ouverture,date_maj_ligne,etat,ministere_tutelle,multi_uai,rpi_concentre,rpi_disperse,code_nature,libelle_nature,Code_type_contrat_prive,PIAL,etablissement_mere,type_rattachement_etablissement_mere,code_

In [19]:
df_commerce = pd.read_csv("./db/fi_etabl_p.csv", sep=";", encoding="utf-8")

df_commerce_bordeaux = df_commerce[df_commerce['nom'] == "Bordeaux"]
df_commerce_bordeaux.to_csv('./db/entreprise_bordeaux.csv', index=False)

C:\Users\darkj\AppData\Local\Temp\ipykernel_33300\88199186.py:1: DtypeWarning: Columns (5,7,12,19,22,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_commerce = pd.read_csv("./db/fi_etabl_p.csv", sep=";", encoding="utf-8")


In [20]:
# df_paris = df[df['code_departement'] == 75]
# # print(df_paris)
# df_paris.to_csv('Paris.csv', index=False)

In [21]:
# df_toulouse = df[df['code_departement'] == 31]
# # print(df_toulouse)
# df_toulouse.to_csv('Toulouse.csv', index=False)

In [22]:
# df_nantes = df[df['code_commune'] == 44109]
# # print(df_nantes)
# df_nantes.to_csv('Nantes.csv', index=False)

In [23]:
# df_angouleme = df[df['code_commune'] == 16015]
# # print(df_angouleme)
# df_angouleme.to_csv('Angouleme.csv', index=False)